In [2]:
import cv2
import mediapipe as mp
import numpy as np

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)
def detect_open_fingers(landmarks, frame):
    finger_landmarks = landmarks.landmark
    open_fingers = []

    # Define landmarks for thumb, index, middle, ring, and pinky fingers
    finger_tip_ids = [4, 8, 12, 16, 20]

    if len(finger_landmarks) >= 21:  # Check if enough landmarks are detected
        for finger_tip_id in finger_tip_ids:
            tip = finger_landmarks[finger_tip_id]
            dip = finger_landmarks[finger_tip_id - 1]
            pip = finger_landmarks[finger_tip_id - 2]

            # Adjust for flipped frame: Check if fingertip (tip) is higher (in y) than PIP and also to the right (in x)
            if tip.x < pip.x and tip.x < dip.x and tip.y < pip.y and tip.y < dip.y:
                if finger_tip_id == 4:
                    open_fingers.append('Thumb')
                else:
                    open_fingers.append(finger_name(finger_tip_id))

    return open_fingers

def finger_name(finger_tip_id):
    finger_names = ['Index', 'Middle', 'Ring', 'Pinky']
    # Calculate the correct index
    return finger_names[(finger_tip_id - 8) // 4]
# Set the frame size to 1280x720 (720p)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            continue

        # Flip the frame horizontally
        frame = cv2.flip(frame, 1)

        # Convert the BGR image to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame and detect hands
        results = hands.process(frame_rgb)

        if results.multi_hand_landmarks:
            for landmarks in results.multi_hand_landmarks:
                # Detect and display the open fingers for thumb and other fingers
                open_fingers = detect_open_fingers(landmarks, frame)
                cv2.putText(frame, f'Open Fingers: {", ".join(open_fingers)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow('Hand Tracking', frame)

        if cv2.waitKey(1) & 0xFF == 27:  # Press 'Esc' to exit
            break

cap.release()
cv2.destroyAllWindows()


